In [ ]:
# install the requirements
!pip install spacy
!python -m spacy download es_core_news_md
!python -m spacy download en_core_web_md

# `spacy`: el *Ruby on Rails* del PLN

[spacy](http://www.spacy.io/) es una librería de procesamiento del lenguaje natural, robusta, rápida, fácil de instalar y utilizar e integrable con [otras librerías de *NLP* y de *deep learning*](https://spacy.io/usage/facts-figures#section-other-libraries). 

Tiene modelos entrenados en varios idiomas y permite realizar las [típicas tareas](https://spacy.io/usage/facts-figures) de segmentación por oraciones, tokenizanción, análisis morfológico, extracción de entidades y análisis de opinión.

Una vez instalados los modelos, podemos importarlos fácilmente:

In [1]:
import spacy

# cargamos el modelo entrenado en español
nlp = spacy.load("es_core_news_md")

In [2]:
texto = """España incumple la regla del déficit del euro y queda como único país bajo el control de Bruselas. 
España no aprobará finalmente la regla europa del déficit y se quedará como único país de la Eurozona que 
suspende y sigue bajo vigilancia. El Gobierno se salta finalmente el requisito de saneamiento presupuestario 
del Tratado de Maastricht y obtiene así más margen de gasto en 2018."""

# y procesamos el texto
doc = nlp(texto)

## Procesando oraciones, palabras y entidades

Podemos iterar fácilmente sobre la lista de oraciones y recorrer los tokens para acceder a su información morfo-sintáctica:

In [3]:
for sentence in doc.sents:
    print("Oración: {}".format(sentence))
    for token in sentence:
        print(
            "{}/{} => etiqueta {}/{} y dependencia {}".format(
                token, token.lemma_, token.pos_, token.tag_, token.dep_
            )
        )

Oración: España incumple la regla del déficit del euro y queda como único país bajo el control de Bruselas. 

España/España => etiqueta PROPN/PROPN___ y dependencia nsubj
incumple/incumplir => etiqueta VERB/VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin y dependencia ROOT
la/lo => etiqueta DET/DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art y dependencia det
regla/reglar => etiqueta NOUN/NOUN__Gender=Fem|Number=Sing y dependencia obj
del/del => etiqueta ADP/ADP__AdpType=Preppron|Gender=Masc|Number=Sing y dependencia case
déficit/déficit => etiqueta NOUN/NOUN__Gender=Masc|Number=Sing y dependencia nmod
del/del => etiqueta ADP/ADP__AdpType=Preppron|Gender=Masc|Number=Sing y dependencia case
euro/euro => etiqueta NOUN/NOUN__Gender=Masc|Number=Sing y dependencia nmod
y/y => etiqueta CONJ/CCONJ___ y dependencia cc
queda/quedo => etiqueta VERB/VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin y dependencia conj
como/comer => etiqueta SCONJ/SCONJ___ y dependencia 

El elemeno `doc` tiene una propiedad `.ents` que permite acceder a las entidades nombradas que hayan sido localizadas:

In [4]:
print(doc.ents)

for entity in doc.ents:
    print("{} es de tipo {}".format(entity, entity.label_))

(Bruselas, España, Eurozona, El Gobierno, Tratado de Maastricht)
Bruselas es de tipo LOC
España es de tipo PER
Eurozona es de tipo LOC
El Gobierno es de tipo MISC
Tratado de Maastricht es de tipo MISC


## Visualizando árboles de dependencias y entidades

Podemos acceder al árbol completo a través del método `.print_tree()`:

In [ ]:
doc.to_json()

Pero también se puede dibujar el grafo con las dependencias:

In [6]:
from spacy import displacy

doc2 = nlp("La niña come manzanas.")

displacy.serve(doc2, style="dep")

/home/victor/miniconda3/envs/kschool/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


O el de las entidades:

In [5]:
from spacy import displacy

displacy.serve(doc, style="ent")

/home/victor/miniconda3/envs/kschool/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [ ]:
texto = """Los ciudadanos españoles son conscientes de que esa salida de Reino Unido no será gratuita. Literalmente. Uno de cada tres teme que el Brexit le cueste caro a la economía española: el 36,9% prevé consecuencias «muy negativas» para sus bolsillos."""
doc = nlp(texto)
displacy.serve(doc, style="ent")

In [2]:
nlp_en = spacy.load("en_core_web_md")

In [8]:
text = """A trade war between the world’s two largest economies officially began on Friday morning as the Trump 
administration followed through with its threat to impose tariffs on $34 billion worth of Chinese products, a 
significant escalation of a fight that could hurt companies and consumers in both the United States and China."""
doc = nlp_en(text)
displacy.serve(doc, style="ent")

/home/victor/miniconda3/envs/kschool/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


## Similitud semántica entre palabras, frases y documentos

spaCy permite [calcular la similitud semántica](https://spacy.io/usage/vectors-similarity) entre cualquier par de objetos de tipo `Doc`, `Span` o `Token`. 

Ojo, La similitud semántica es un concepto algo subjetivo, pero en este caso se puede entender como la probabilidad de que dos palabras aparezcan en los mismos contextos.

In [6]:
# analizamos algunas colocaciones en inglés
token1, _, token2 = nlp_en("cats and dogs")
token3, _, token4 = nlp_en("research and development")

# medimos la similitud semántica entre algunos pares
print(token1, "vs", token2, token1.similarity(token2))
print(token3, "vs", token4, token3.similarity(token4))
print(token1, "vs", token4, token1.similarity(token4))

cats vs dogs 0.83117634
research vs development 0.58287174
cats vs development 0.104666404


In [9]:
token1.vector

array([-0.26763  ,  0.029846 , -0.3437   , -0.54409  , -0.49919  ,
        0.15928  , -0.35278  , -0.2036   ,  0.23482  ,  1.5671   ,
       -0.36458  , -0.028713 , -0.27053  ,  0.2504   , -0.18126  ,
        0.13453  ,  0.25795  ,  0.93213  , -0.12841  , -0.18505  ,
       -0.57597  ,  0.18538  , -0.19147  , -0.38465  ,  0.21656  ,
       -0.4387   , -0.27846  , -0.41339  ,  0.37859  , -0.2199   ,
       -0.25907  , -0.019796 , -0.31885  ,  0.12921  ,  0.22168  ,
        0.32671  ,  0.46943  , -0.81922  , -0.20031  ,  0.013561 ,
       -0.14663  ,  0.14438  ,  0.0098044, -0.15439  ,  0.21146  ,
       -0.28409  , -0.4036   ,  0.45355  ,  0.12173  , -0.11516  ,
       -0.12235  , -0.096467 , -0.26991  ,  0.028776 , -0.11307  ,
        0.37219  , -0.054718 , -0.20297  , -0.23974  ,  0.86271  ,
        0.25602  , -0.3064   ,  0.014714 , -0.086497 , -0.079054 ,
       -0.33109  ,  0.54892  ,  0.20076  ,  0.28064  ,  0.037788 ,
        0.0076729, -0.0050123, -0.11619  , -0.23804  ,  0.3302

In [4]:
# ¿qué tal funciona en español?
token1, _, token2 = nlp("perros y gatos")
token3, _, token4 = nlp("investigación y desarrollo")

# medimos la similitud semántica entre algunos pares
print(token1, "vs", token2, token1.similarity(token2))
print(token3, "vs", token4, token3.similarity(token4))
print(token1, "vs", token4, token1.similarity(token4))

perros vs gatos 0.91161263
investigación vs desarrollo 0.7544771
perros vs desarrollo 0.346996


In [5]:
token1.vector

array([ 0.420254, -0.660067, -1.545925, -0.965525,  0.193724,  0.209681,
       -0.156194, -0.229921,  0.124098,  1.055757, -0.269885,  0.190388,
       -0.264621, -0.94836 ,  2.631071,  0.358889, -0.037149,  0.886237,
       -0.692818,  0.749372,  0.640033,  0.619242, -0.67352 , -0.917047,
       -1.32988 ,  0.793474, -0.156792,  1.475654,  1.30683 ,  1.04421 ,
        0.703801,  0.171951, -0.888882, -0.08564 ,  0.421157,  0.705225,
        0.958168, -0.150986, -0.034823, -0.701907,  0.285303, -0.074543,
        0.872631, -1.354706,  0.659176,  0.177278, -0.078645, -0.654988,
       -0.652848, -0.31781 ], dtype=float32)

In [18]:
frases = [
    "I love pasta and pizza.",
    "I hate sushi and ramen.",
    "My sister was born in Germany.",
    "I have a Korean car.",
]

frases_vectorizadas = [nlp_en(frase) for frase in frases]

In [19]:
doc = nlp_en("I own a BMW")

for f in frases_vectorizadas:
    print(f, doc.similarity(f))

I love pasta and pizza. 0.654512423292579
I hate sushi and ramen. 0.629948049249409
My sister was born in Germany. 0.6609344769005584
I have a Korean car. 0.8645187837467031
